In [81]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
import os.path
import json

import time
import base64
import jsonpickle

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import base64
from email.message import EmailMessage

import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.modify"]

load_dotenv(find_dotenv())
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [82]:
creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            "credentials.json", SCOPES
        )
        creds = flow.run_local_server(port=4000)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
        token.write(creds.to_json())

service = None
try:
    # Call the Gmail API
    service = build("gmail", "v1", credentials=creds)

except HttpError as error:
# TODO(developer) - Handle errors from gmail API.
    print(f"An error occurred: {error}")

In [83]:
class Email:
    def __init__(self, api_message_obj):
        self.id = api_message_obj['id']
        self.thread_id = api_message_obj['threadId']
        self.snippet = api_message_obj['snippet']
        if 'parts' in api_message_obj['payload']:
            if api_message_obj['payload']['parts'][0]['mimeType'] == 'multipart/alternative':
                message_raw = api_message_obj['payload']['parts'][0]['parts'][0]['body']['data']   
            else:
                message_raw = api_message_obj['payload']['parts'][0]['body']['data']  
        else:
            message_raw = api_message_obj['payload']['body']['data']
        self.body = base64.urlsafe_b64decode(message_raw.encode('UTF8'))
        headers_to_save = []
        for header in api_message_obj['payload']['headers']:
            if header['name'] == 'To' or header['name'] == 'From' or header['name'] == 'Subject' or header['name'] == 'Date':
                headers_to_save.append(header)
            
        self.header = headers_to_save

    def to_JSON(self):
        return jsonpickle.encode(self)
    

In [84]:
def get_email_list():
    try:
        results = service.users().messages().list(userId="me", maxResults=3).execute()
        messages = results.get("messages", [])

        if not messages:
            print("No messages found.")
            return
        for index, message in enumerate(messages):
            api_obj = service.users().messages().get(userId='me', id=message['id']).execute()
            message_obj = Email(api_obj)
            messages[index] = message_obj
        print("Messages retrieved.")
        return messages

    except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
        print(f"An error occurred: {error}")


def send_email(content, to, sender, subject):
  """Create and send an email message
  Returns: Message object, including message id

  Load pre-authorized user credentials from the environment.
  """
  creds, _ = google.auth.default()

  try:
    message = EmailMessage()

    message.set_content(content)

    message["To"] = to
    message["From"] = sender
    message["Subject"] = subject

    # encoded message
    encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

    create_message = {"raw": encoded_message}
    # pylint: disable=E1101
    send_message = (
        service.users()
        .messages()
        .send(userId="me", body=create_message)
        .execute()
    )
    return send_message
  except HttpError as error:
    print(f"An error occurred: {error}")
    send_message = None
    

In [85]:
# Assistant class that can make Gmail API calls
class GmailHelper:
    thread_id = None
    assistant_id = "asst_DJXRcO2u2mAIKMjMoEXbJk6S"
    #"asst_p01Djp64FjuGUIX9kf72DyTe"

    def __init__(self):
        self.client = client
        self.model = "gpt-4"
        self.assistant = None
        self.thread = None
        self.run = None

        if GmailHelper.assistant_id:
            self.assistant = self.client.beta.assistants.retrieve(assistant_id=GmailHelper.assistant_id)

        if GmailHelper.thread_id:
            self.thread = self.client.beta.threads.retrieve(thread_id=GmailHelper.thread_id)

    def create_assistant(self, name, instructions, tools):
        if not self.assistant:
            assistant_obj = self.client.beta.assistants.create(
                name=name,
                instructions=instructions,
                tools=tools,
                model=self.model
            )
            GmailHelper.assistant_id = assistant_obj.id
            self.assistant = assistant_obj
            print(f"Assistant created with ID: {self.assistant.id}")

    def create_thread(self):
        if not self.thread:
            thread_obj = self.client.beta.threads.create()
            GmailHelper.thread_id = thread_obj.id
            self.thread = thread_obj
            print(f"Thread created with ID: {self.thread.id}")

    def add_message_to_thread(self, role, content):
        if self.thread:
            self.client.beta.threads.messages.create(
                thread_id = self.thread.id,
                role = role,
                content = content
            )
    
    def run_assistant(self, instructions):
        if self.thread and self.assistant:
            self.run = self.client.beta.threads.runs.create(
                thread_id = self.thread_id,
                assistant_id = self.assistant_id,
                instructions = instructions
            )

    def print_thread(self):
        if self.thread:
            messages = self.client.beta.threads.messages.list(thread_id = self.thread_id)

            for message in reversed(list(messages)):
                role = message.role
                content = message.content[0].text.value
                print(role + ': ' + content)

    def call_required_functions(self, required_actions):
        if not self.run:
            return
        tool_outputs = []

        print(required_actions)

        for action in required_actions["tool_calls"]:
            func_name = action["function"]["name"]
            arguments = json.loads(action["function"]["arguments"])
            output = []

            if func_name == "get_email_list":
                output = get_email_list()
            elif func_name == "send_email":
                print(arguments)
                output = send_email(content=arguments["content"], to=arguments["to"], sender=arguments["sender"], subject=arguments["subject"])

            final_str = ""
            for item in output:
                if isinstance(item, Email):
                    final_str += "".join(item.to_JSON())
                else:
                    final_str += "".join(item)

            print(f"Tool output: {output}")
            tool_outputs.append({"tool_call_id": action["id"], "output": final_str})

        self.client.beta.threads.runs.submit_tool_outputs(
            thread_id = self.thread.id,
            run_id = self.run.id,
            tool_outputs = tool_outputs
        )
    
    def wait_for_completion(self):
        if self.thread and self.run:
            while True:
                time.sleep(5)
                run_status = self.client.beta.threads.runs.retrieve(
                    thread_id = self.thread.id,
                    run_id = self.run.id
                )

                if run_status.status == "completed":
                    break
                elif run_status.status == "requires_action":
                    self.call_required_functions(required_actions=run_status.required_action.submit_tool_outputs.model_dump())

    def run_steps(self):
        run_steps = self.client.beta.threads.runs.steps.list(
            thread_id = self.thread.id,
            run_id = self.run.id
        )
        print(f"Run steps: {run_steps}")
        return run_steps.data


In [86]:
manager = GmailHelper()

In [87]:
manager.create_assistant(
    name = "Gmail Helper",
    instructions = "Your go-to assistant for reading and editing emails. Gmail Helper is a personal assistant dedicated to reading and sending emails, primarily through Google Gmail API integration. Its core functionality includes checking the user’s email to read and summarize emails they receive. It is capable of sending emails to the user's contacts but will always seek confirmation from the user before doing so, ensuring that the user has control over what they send. Gmail Helper maintains a professional demeanor, focusing on providing clear, concise, and accurate scheduling information. It prioritizes user privacy and confidentiality.The assistant is also equipped to guide users through the Google Gmail API’s functionalities and troubleshoot common issues, using language tailored to the user’s level of technical expertise. Overall, Gmail Helper adopts a polite, efficient, and accommodating tone, aiming to make email management and event scheduling as smooth and user-friendly as possible.",
    tools = [{
      "type": "function",
      "function": {
        "name": "get_email_list",
        "description": "Get a list of the user's mailbox contents",
      }
    }, {
      "type": "function",
      "function": {
          "name": "send_email",
        "description": "Send an email to another user",
        "parameters": {
        "type": "object",
        "properties": {
          "content": {"type": "string", "description": "The content of the email to be sent"},
          "to": {"type": "string", "description": "Email address of email recipient"},
          "sender": {"type": "string", "description": "The sender of the email, namely the user"},
          "subject": {"type": "string", "description": "The subject of the email being sent"}
        },
        "required": ["content", "to", "sender", "subject"]
        }
      }
  }]
)

manager.create_thread()

manager.add_message_to_thread(
    role = "user",
    content = "What are my latest emails?"
)

manager.run_assistant(instructions="Get the user's latest emails that they have received")
manager.wait_for_completion()

manager.print_thread()

Thread created with ID: thread_wisn1doBXFnqwzojI8DRAb5L
{'tool_calls': [{'id': 'call_LHoZ81j2ubeLlYwpB4OP5hrh', 'function': {'arguments': '{}', 'name': 'get_email_list'}, 'type': 'function'}]}
Messages retrieved.
Tool output: [<__main__.Email object at 0x113ce0820>, <__main__.Email object at 0x113b95f60>, <__main__.Email object at 0x113b97a90>]
user: What are my latest emails?
assistant: Here are your latest emails:

Email 1:
- From: user3 <ersptester3@gmail.com>
- Date: Mon, 8 Apr 2024 11:40:23 -0700
- Subject: Hello2
- Snippet: Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. A erat nam at lectus urna duis convallis convallis. Cursus mattis molestie a.

Email 2:
- From: Not specified
- Date: Wed, 21 Feb 2024 18:51:35 -0800
- Subject: test
- Snippet: this is a test

Email 3:
- From: Google <no-reply@google.com>
- Date: Thu, 22 Feb 2024 02:50:36 GMT
- Subject: Your Google Account was recovered successfully
- Sni

In [88]:
""" manager.add_message_to_thread(
    role = "user",
    content = "From my email, send a message to ersptester2@gmail.com with the subject 'URGENT' and the content 'You have been hecked'"
)

manager.run_assistant(instructions="Send an email from the user using the subject, content, and recipient specified by the user.")
manager.wait_for_completion()
manager.print_thread() """

' manager.add_message_to_thread(\n    role = "user",\n    content = "From my email, send a message to ersptester2@gmail.com with the subject \'URGENT\' and the content \'You have been hecked\'"\n)\n\nmanager.run_assistant(instructions="Send an email from the user using the subject, content, and recipient specified by the user.")\nmanager.wait_for_completion()\nmanager.print_thread() '

In [89]:
manager.add_message_to_thread(
    role = "user",
    content = "Show me the content of the first email"
)

manager.run_assistant(instructions="Show the user the email's content")
manager.wait_for_completion()
manager.print_thread()

user: What are my latest emails?
assistant: Here are your latest emails:

Email 1:
- From: user3 <ersptester3@gmail.com>
- Date: Mon, 8 Apr 2024 11:40:23 -0700
- Subject: Hello2
- Snippet: Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. A erat nam at lectus urna duis convallis convallis. Cursus mattis molestie a.

Email 2:
- From: Not specified
- Date: Wed, 21 Feb 2024 18:51:35 -0800
- Subject: test
- Snippet: this is a test

Email 3:
- From: Google <no-reply@google.com>
- Date: Thu, 22 Feb 2024 02:50:36 GMT
- Subject: Your Google Account was recovered successfully
- Snippet: Account recovered successfully ersptester3@gmail.com Welcome back to your account If you suspect you were locked out of your account because of changes made by someone else, you should review &.
user: Show me the content of the first email
assistant: Here is the content of your first email:

Lorem ipsum dolor sit amet, consectetur adipisci

In [90]:
service.close()